In [13]:
import glob
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

# for warnings
import warnings
warnings.filterwarnings("ignore")

imgs = glob.glob("./img/*.png")

# img size
width = 250
height = 100

X = []
Y = []

for img in imgs:
    
    fileName = os.path.basename(img)
    label = fileName.split('_')[0] 
    
    # "L" for grayscale. normalized with 255
    im = np.array(Image.open(img).convert("L").resize((width, height))) / 255
    X.append(im)
    Y.append(label)

X = np.array(X)
X = X.reshape(X.shape[0], width, height, 1)  # 1 is channel

def onehotLabels(values):
    labelEncoder = LabelEncoder()
    integerEncoded = labelEncoder.fit_transform(values)
    onehotEncoder = OneHotEncoder(sparse=False)
    integerEncoded = integerEncoded.reshape(len(integerEncoded), 1)
    onehot_encoded = onehotEncoder.fit_transform(integerEncoded)
    return onehot_encoded

Y = onehotLabels(Y)

#train_test_split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.25, random_state = 2)  



In [19]:
# CNN Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(width, height, 1)))
model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(3, activation="softmax"))      

model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics= ["accuracy"])  

model.fit(trainX, trainY, epochs=35, batch_size=64)


Epoch 1/35
1/1 [==============================] - 2s 2s/step - loss: 1.1565 - accuracy: 0.3333
Epoch 2/35
1/1 [==============================] - 1s 611ms/step - loss: 18.5898 - accuracy: 0.4444
Epoch 3/35
1/1 [==============================] - 1s 620ms/step - loss: 31.4373 - accuracy: 0.3889
Epoch 4/35
1/1 [==============================] - 1s 608ms/step - loss: 23.6892 - accuracy: 0.1111
Epoch 5/35
1/1 [==============================] - 1s 627ms/step - loss: 7.8250 - accuracy: 0.2778
Epoch 6/35
1/1 [==============================] - 1s 617ms/step - loss: 4.1815 - accuracy: 0.6667
Epoch 7/35
1/1 [==============================] - 1s 611ms/step - loss: 4.3937 - accuracy: 0.5000
Epoch 8/35
1/1 [==============================] - 1s 609ms/step - loss: 3.8486 - accuracy: 0.5000
Epoch 9/35
1/1 [==============================] - 1s 662ms/step - loss: 2.0750 - accuracy: 0.3889
Epoch 10/35
1/1 [==============================] - 1s 643ms/step - loss: 2.1263 - accuracy: 0.4444
Epoch 11/35
1/1 [==

In [20]:
# import tensorflow as tf
# from tensorflow.keras.utils import to_categorical

# # Assuming trainY contains class labels as integers (e.g., [0, 1, 2, ...])
# # Convert the class labels to one-hot encoded format
# trainY_onehot = to_categorical(trainY, num_classes=3)
# testY_onehot = to_categorical(testY, num_classes=3)

# # Now, train the model with the one-hot encoded target labels
# model.fit(trainX, trainY_onehot, epochs=35, batch_size=64)

In [22]:

# Load Trained Weights
if os.path.exists("./trex_weight.h5"):
    model.load_weights("trex_weight.h5")
    print("Weights loaded.") 
# ############

# score_train = model.evaluate(trainX, trainY_onehot)
score_train = model.evaluate(trainX, trainY)
print("Training Accuracy: %",score_train[1]*100)    
    
score_test = model.evaluate(testX, testY)
print("Test Accuracy: %",score_test[1]*100)  


open("model_new.json","w").write(model.to_json())
model.save_weights("trex_weight_new.h5")   


1/1 [==============================] - 0s 174ms/step - loss: 9.9944e-04 - accuracy: 1.0000
Training Accuracy: % 100.0
1/1 [==============================] - 0s 133ms/step - loss: 0.2123 - accuracy: 0.8571
Test Accuracy: % 85.71428656578064
